<a href="https://colab.research.google.com/github/preethamnagesh8/captcha_recognition/blob/main/Security_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import pandas as pd
from torchvision import models
import torch.nn as nn
from fastai.vision.all import Path
import torch
from torch import Tensor

In [3]:
NUMBER = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
ALPHABET = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
ALL_CHAR_SET = NUMBER + ALPHABET
ALL_CHAR_SET_LEN = len(ALL_CHAR_SET)
MAX_CAPTCHA = 5

In [4]:
def encode(a):
    onehot = [0]*ALL_CHAR_SET_LEN
    idx = ALL_CHAR_SET.index(a)
    onehot[idx] += 1
    return onehot

In [5]:
class Mydataset(Dataset):
    def __init__(self, path, is_train=True, transform=None):
        self.path = path
        if is_train: self.img = os.listdir(self.path)[:1000]
        else: self.img = os.listdir(self.path)[1001:]
        try: self.img.remove('3bnfnd.png')
        except: pass
        self.transform = transform
        
    def __getitem__(self, idx):
        img_path = self.img[idx]
        img = Image.open(self.path/img_path)
        img = img.convert('L')
        label = Path(self.path/img_path).name[:-4]
        label_oh = []
        for i in label:
            label_oh += encode(i)
        if self.transform is not None:
            img = self.transform(img)
        return img, np.array(label_oh), label
    
    def __len__(self):
        return len(self.img)

In [6]:
transform = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
])

In [8]:
train_ds = Mydataset(Path('/home/samples'), transform=transform)
test_ds = Mydataset(Path('/home/samples'), False, transform)
train_dl = DataLoader(train_ds, batch_size=64, num_workers=0)
test_dl = DataLoader(train_ds, batch_size=1, num_workers=0)

In [9]:
model = models.resnet18(pretrained=False)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [10]:
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [11]:
model.fc = nn.Linear(in_features=512, out_features=ALL_CHAR_SET_LEN*MAX_CAPTCHA, bias=True)

In [12]:
model.cuda();

In [13]:
loss_func = nn.MultiLabelSoftMarginLoss()
optm = torch.optim.Adam(model.parameters(), lr=0.001)

In [14]:
for epoch in range(20):
    for step, i in enumerate(train_dl):
        img, label_oh, label = i
        img = Tensor(img).cuda()
        label_oh = Tensor(label_oh.float()).cuda()
        pred = model(img)
        loss = loss_func(pred, label_oh)
        optm.zero_grad()
        loss.backward()
        optm.step()
        print('eopch:', epoch+1, 'step:', step+1, 'loss:', loss.item())

eopch: 1 step: 1 loss: 0.7314121127128601
eopch: 1 step: 2 loss: 0.5252151489257812
eopch: 1 step: 3 loss: 0.3520660698413849
eopch: 1 step: 4 loss: 0.23997244238853455
eopch: 1 step: 5 loss: 0.1759261041879654
eopch: 1 step: 6 loss: 0.1441778838634491
eopch: 1 step: 7 loss: 0.12949956953525543
eopch: 1 step: 8 loss: 0.1262603998184204
eopch: 1 step: 9 loss: 0.1259596049785614
eopch: 1 step: 10 loss: 0.12621787190437317
eopch: 1 step: 11 loss: 0.1283486783504486
eopch: 1 step: 12 loss: 0.12822748720645905
eopch: 1 step: 13 loss: 0.12705940008163452
eopch: 1 step: 14 loss: 0.12577812373638153
eopch: 1 step: 15 loss: 0.12409727275371552
eopch: 1 step: 16 loss: 0.12358000129461288
eopch: 2 step: 1 loss: 0.11840108782052994
eopch: 2 step: 2 loss: 0.11522053182125092
eopch: 2 step: 3 loss: 0.11535993218421936
eopch: 2 step: 4 loss: 0.11394261568784714
eopch: 2 step: 5 loss: 0.11417092382907867
eopch: 2 step: 6 loss: 0.11316531896591187
eopch: 2 step: 7 loss: 0.11325754225254059
eopch: 2 ste

In [15]:
import torch
import gc
# del variables
gc.collect()
torch.cuda.empty_cache()

In [16]:
model.eval()

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [17]:
for step, (img, label_oh, label) in enumerate(test_dl):
    img = Tensor(img).cuda()
    pred = model(img)

    c0 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c = '%s%s%s%s%s' % (c0, c1, c2, c3, c4)

    print('label:', label[0], 'pred:', c)

label: 8bbw8 pred: 8bbw8
label: yg5bb pred: yg5bb
label: ewcf5 pred: ewcf5
label: npxb7 pred: npxb7
label: 658xe pred: 658xe
label: mw5p2 pred: mw5p2
label: yf424 pred: yf424
label: gfxcc pred: gfxcc
label: 4d22m pred: 4d22m
label: 8gecm pred: 8gecm
label: yyn57 pred: yyn57
label: cdf77 pred: cdf77
label: 68x48 pred: 68x48
label: x37bf pred: x37bf
label: w7e6m pred: w7e6m
label: nf8b8 pred: nf8b8
label: 47e4p pred: 47e4p
label: nwfde pred: nwfde
label: e6m6p pred: e6m6p
label: nbf8m pred: nbf8m
label: pcmcc pred: pcmcc
label: 4c8n8 pred: 4c8n8
label: p57fn pred: p57fn
label: 573d8 pred: 573d8
label: 42xpy pred: 42xpy
label: n3ffn pred: n3ffn
label: d3c8y pred: d3c8y
label: wddcp pred: wddcp
label: gbxyy pred: gbxyy
label: mpmy5 pred: mpmy5
label: 2mpnn pred: 2mpnn
label: dyp7n pred: dyp7n
label: bd3b7 pred: bd3b7
label: ncw4g pred: ncw4g
label: 62nb3 pred: 62nb3
label: 33f7m pred: 33f7m
label: c2fb7 pred: c2fb7
label: 3w2bw pred: 3w2bw
label: xymfn pred: xymfn
label: nnf8b pred: nnf8b
